# Notebook 01 – Veri Keşfi ve Tanımlayıcı İstatistik

Bu notebook’ta elektrik tüketim (Tahakkuk) ve ödeme (Tahsilat) verileri
incelenerek veri yapısı, veri kalitesi ve temel istatistikler analiz edilecektir.


In [153]:
# Gerekli kütüphaneleri import et
import pandas as pd
import numpy as np

### Veri Setinin Yüklenmesi

Bu adımda analizde kullanılacak ham veri dosyaları okunmuştur.
Veriler farklı kaynaklardan ve dönemlerden geldiği için, ilerleyen aşamalarda birleştirme ve standartlaştırma işlemlerine
tabi tutulacaktır.

In [154]:
# Excel dosyasını yükle
file_path = "elektrik_veri_hashed.xlsx"

xls = pd.ExcelFile(file_path)
xls.sheet_names
# Sayfa isimlerini görüntüle
print("Sayfa isimleri:", xls.sheet_names)

Sayfa isimleri: ['Tahsilat', 'Tahsilat 1', 'Tahakkuk', 'Tahakkuk 1', 'Tahakkuk 2']


Veri seti 5 farklı sayfadan oluşmakta olup, tüketim verileri 3 ayrı Tahakkuk sayfasında tutulmaktadır.

In [155]:
# Her sayfayı ayrı DataFrame'e yükle
df_tahsilat = pd.read_excel(xls, sheet_name='Tahsilat')
df_tahsilat_1 = pd.read_excel(xls, sheet_name='Tahsilat 1')
df_tahakkuk = pd.read_excel(xls, sheet_name='Tahakkuk') # Hamamözü
df_tahakkuk_1 = pd.read_excel(xls, sheet_name='Tahakkuk 1') # Gümüşhacıköy
df_tahakkuk_2 = pd.read_excel(xls, sheet_name='Tahakkuk 2') # Göynücek


In [99]:
# Veri setlerindeki kayıt sayılarını
print(
    f"Tahsilat: {df_tahsilat.shape[0]} kayıt \n "
    f"Tahsilat 1: {df_tahsilat_1.shape[0]} kayıt \n "
    f"Tahakkuk: {df_tahakkuk.shape[0]} kayıt \n "
    f"Tahakkuk 1: {df_tahakkuk_1.shape[0]} kayıt \n "
    f"Tahakkuk 2: {df_tahakkuk_2.shape[0]} kayıt "
)

Tahsilat: 636993 kayıt 
 Tahsilat 1: 917632 kayıt 
 Tahakkuk: 1185698 kayıt 
 Tahakkuk 1: 765657 kayıt 
 Tahakkuk 2: 295223 kayıt 


Bu bölümde veri setinin genel yapısı incelenmiştir.
Satır ve sütun sayıları, veri tipleri ve eksik değer durumu
kontrol edilerek verinin analiz için uygunluğu değerlendirilmiştir.

In [44]:
# Veri setlerinin boyutları
print(f"Tahsilat: {df_tahsilat.shape} | "
      f"Tahsilat 1: {df_tahsilat_1.shape} | "
      f"Tahakkuk: {df_tahakkuk.shape} | "
      f"Tahakkuk 1: {df_tahakkuk_1.shape} | "
      f"Tahakkuk 2: {df_tahakkuk_2.shape}")

Tahsilat: (636993, 9) | Tahsilat 1: (917632, 22) | Tahakkuk: (124818, 10) | Tahakkuk 1: (765657, 10) | Tahakkuk 2: (295223, 10)


In [156]:
# Tahakkuk veri setlerinin ilgili ilçelere göre isimlendirilmesi ve atanması
df_hamamozu = df_tahakkuk
df_gumushacikoy = df_tahakkuk_1
df_goynucek = df_tahakkuk_2

In [23]:
# first 5 rows of the data
df_tahsilat.head()

,Şube,Kasa,İlçe,Söz.hsp.(bağımsız),Tahsilat Tarihi,Nakit Tahsilat,Mahsuben Tahsilat,Kredi Kartı Tahsilatı,Banka Tahsilatı
0,Tayin edilmedi,Tayin edilmedi,TAŞOVA,4989745446,2023-11-06,NaN,8648.95,NaN,NaN
1,Tayin edilmedi,Tayin edilmedi,TAŞOVA,4989745446,2024-06-26,NaN,762.40,NaN,NaN
2,Tayin edilmedi,Tayin edilmedi,TAŞOVA,4989745446,2024-07-10,NaN,311.60,NaN,NaN
3,PTT,PTT/PV,TAŞOVA,4254955886,2023-01-19,NaN,NaN,NaN,130.5
4,PTT,PTT/PV,TAŞOVA,4254955886,2023-02-17,NaN,NaN,NaN,117.0


In [70]:
# structural information of the dataset
df_tahsilat.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636993 entries, 0 to 636992
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Şube                   636993 non-null  object        
 1   Kasa                   636993 non-null  object        
 2   İlçe                   636993 non-null  object        
 3   Söz.hsp.(bağımsız)     636993 non-null  int64         
 4   Tahsilat Tarihi        636993 non-null  datetime64[ns]
 5   Nakit Tahsilat         523 non-null     float64       
 6   Mahsuben Tahsilat      7542 non-null    float64       
 7   Kredi Kartı Tahsilatı  0 non-null       float64       
 8   Banka Tahsilatı        628933 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(3)
memory usage: 43.7+ MB


### Veri Tiplerinin Düzenlenmesi

Analiz doğruluğunu ve performansı artırmak amacıyla,
değişkenlerin veri tipleri iş anlamlarına uygun şekilde
yeniden düzenlenmiştir.

- Kimlik ve kod niteliğindeki alanlar metin (object) olarak,
- Sınırlı ve tekrarlı değerler içeren alanlar kategorik (categorical),
- Tarih alanları datetime formatında,
- Sayısal ölçümler ise float veri tipiyle ele alınmıştır.

In [88]:
# Tahsilat verilerinin şube bazında dağılımı ve kayıt sayılarının analizi
df_tahsilat.value_counts(["Şube"])

Şube               
AKTİFBANK              193135
ZİRAAT                 117521
İŞBANK                  87383
HALKBANK                60890
PTT                     48760
Y.KREDİ                 22528
GARANTİ                 21221
AKBANK                  20618
VAKIFLAR                14900
DENİZBANK               12366
FİNANSBANK              10134
T. FİNANS                7709
Tayin edilmedi           7542
TEB                      3944
KUVEYTTÜRK               3077
ZİRAAT KATILIM           1510
ALBARAKA                 1096
ING BANK                  989
ŞEKERBANK                 701
AKTİFBANK-BAYİ            518
AKTİFBANK-SANAL POS       305
VAKIF KATILIM              64
HSBC                       44
ANADOLUBANK                30
ICBC TURKEY                 5
ABANK                       3
Name: count, dtype: int64

In [119]:
# Tahsilat verilerinin kasa bazında dağılımı ve kayıt sayılarının analizi
df_tahsilat.value_counts(["Kasa"]) 

Kasa  
AKT/PN    168183
ZIR/PO     62390
TIS/PO     58423
ZIR/PV     53212
PTT/PV     48760
           ...  
ICB/PI         2
HSB/PI         2
FIN/PT         2
ALT/PI         1
VKF/PT         1
Name: count, Length: 76, dtype: int64

In [89]:
# Tahsilat verilerinin ilçe bazında dağılımı ve kayıt sayılarının analizi
df_tahsilat.value_counts(["İlçe"])

İlçe        
TAŞOVA          289077
GÜMÜŞHACIKÖY    252818
GÖYNÜCEK         82519
HAMAMÖZÜ         12579
Name: count, dtype: int64

Tahsilatların yapıldığı ilçeler **Taşova , Gümüşhacıköy, Göynücek, Hamamözü** 'dür.

In [157]:
# Belirlenen sütunların bellek verimliliği için kategori tipine dönüştürülmesi
Cat_sutunlar = [
    "Şube",
    "Kasa",
    "İlçe",
]

df_tahsilat[Cat_sutunlar] = df_tahsilat[Cat_sutunlar].apply(lambda x: x.astype("category"))

**Not:** Tahsilat tablosundaki **Şube**, **Kasa** ve **İlçe** kolonları
kategorik değişken olarak düzenlenmiştir.

In [158]:
# Sözleşme hesap numaralarının metinsel veri tipine dönüştürülmesi
df_tahsilat["Söz.hsp.(bağımsız)"] = df_tahsilat["Söz.hsp.(bağımsız)"].astype(str) 

**Not:** Tahsilat verisinde yer alan **Sözleşme hesap numarası**
değişkeni, sayısal bir büyüklüğü temsil etmekten ziyade
**benzersiz bir tanımlayıcı (ID)** niteliği taşımaktadır.
Bu nedenle, bu değişkenin sayısal işlemlere dahil edilmesini
önlemek ve veri tipinden kaynaklanabilecek hataların önüne
geçmek amacıyla metinsel (string) veri tipine dönüştürülmüştür.

Bu dönüşüm, grup bazlı analizler, birleştirme (merge) işlemleri
ve benzersiz müşteri sayımı gibi adımlarda daha tutarlı ve
güvenilir sonuçlar elde edilmesini sağlamaktadır.


### Tanımlayıcı İstatistikler

Sayısal değişkenler için tanımlayıcı istatistikler hesaplanmıştır.
Bu aşamada dağılım yapısı, merkezi eğilim ölçüleri
ve uç değerlerin varlığı gözlemlenmiştir.

Tarih değişkenleri için yalnızca kapsanan dönem aralığı
analitik olarak anlamlı kabul edilmiştir.

In [139]:
# Generate descriptive statistics for the DataFrame
df_tahsilat.describe().round(2).T 

,count,mean,min,25%,50%,75%,max,std
Tahsilat Tarihi,636993,2024-03-05 09:26:07.644856320,2023-01-01 00:00:00,2023-07-28 00:00:00,2024-02-26 00:00:00,2024-09-30 00:00:00,2025-05-31 00:00:00,NaN
Nakit Tahsilat,523.0,694.97,7.45,425.33,524.67,688.83,11373.74,758.32
Mahsuben Tahsilat,7542.0,6180.18,-34508.95,44.48,290.41,2729.11,399526.78,23828.02
Kredi Kartı Tahsilatı,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Banka Tahsilatı,628933.0,372.63,0.01,120.0,208.0,322.0,606473.8,3265.43


Tahsilat verilerinin zaman aralığı incelendiğinde,
kayıtların **1 Ocak 2023 ile 31 Mayıs 2025 tarihleri arasını**
kapsadığı görülmektedir.

**Banka tahsilatları, 628.933 işlem** ile en yüksek işlem sayısına
sahip tahsilat türüdür. Banka tahsilatlarında ortalama işlem
tutarı 372,63 TL olup, medyan değer 208 TL seviyesindedir.
Bununla birlikte maksimum tutarın 606.473 TL’ye kadar çıkması,
az sayıda yüksek tutarlı işlemin dağılımı genişlettiğini
göstermektedir. Standart sapmanın 3.265 TL olması, banka
tahsilatlarında tutarların görece değişken olduğunu ortaya koymaktadır.

**Nakit tahsilatlarda işlem** sayısı oldukça sınırlı olup **523 işlem**
gerçekleşmiştir. Ortalama tahsilat tutarı 694,97 TL, medyan
tutar ise 524,67 TL’dir. Maksimum tutarın 11.373 TL olması,
nakit tahsilatların daha çok orta ve yüksek tutar aralıklarında
yoğunlaştığını göstermektedir.

**Mahsuben tahsilatlarda 7.542 işlem** bulunmaktadır. Ortalama
tahsilat tutarı 6.180 TL olmakla birlikte, minimum değerin
-34.508 TL olması ve maksimum değerin 399.526 TL’ye ulaşması,
bu tahsilat türünde düzeltme, iptal ve mahsuplaşma işlemlerinin
önemli yer tuttuğuna işaret etmektedir. Standart sapmanın
23.828 TL gibi yüksek bir seviyede olması, bu tahsilat türünde
tutar dağılımının oldukça geniş olduğunu göstermektedir.

**Kredi kartı tahsilatına ait kayıt bulunmaması**, ilgili dönemde
bu ödeme kanalının veri setinde yer almadığını ya da aktif
olarak kullanılmadığını göstermektedir.

*Genel olarak değerlendirildiğinde, tahsilat verilerinde tutar
dağılımının sağa çarpık olduğu ve özellikle banka ve mahsuben
tahsilatlarda az sayıda yüksek tutarlı işlemin ortalama değerler
üzerinde belirleyici olduğu sonucuna varılmıştır.*

In [87]:
# Tahsilat yöntemlerinin doluluk oranlarına göre yüzdesel dağılımının hesaplanması
print(
    f"Banka Tahsilat        : %{(df_tahsilat['Banka Tahsilatı'].notna().sum() / len(df_tahsilat) * 100):.1f}\n"
    f"Mahsuben Tahsilat     : %{(df_tahsilat['Mahsuben Tahsilat'].notna().sum() / len(df_tahsilat) * 100):.1f}\n"
    f"Nakit Tahsilat        : %{(df_tahsilat['Nakit Tahsilat'].notna().sum() / len(df_tahsilat) * 100):.1f}\n"
    f"Kredi Kartı Tahsilat  : %{(df_tahsilat['Kredi Kartı Tahsilatı'].notna().sum() / len(df_tahsilat) * 100):.1f}"
)

Banka Tahsilat        : %98.7
Mahsuben Tahsilat     : %1.2
Nakit Tahsilat        : %0.1
Kredi Kartı Tahsilat  : %0.0


Ödemelerin yüzde **98.7** banka tahsilatı ile gerçekleştirilmektedir.

In [172]:
# reaching missing observations
df_tahsilat.isna().sum()

Şube                          0
Kasa                          0
İlçe                          0
Söz.hsp.(bağımsız)            0
Tahsilat Tarihi               0
Nakit Tahsilat           636470
Mahsuben Tahsilat        629451
Kredi Kartı Tahsilatı    636993
Banka Tahsilatı            8060
dtype: int64

Tahsilat verisindeki eksik değerler incelendiğinde,
kimlik ve zaman bilgisi içeren sütunlarda
(**Şube**, **Kasa**, **İlçe**, **Söz.hsp.(bağımsız)** ve
**Tahsilat Tarihi**) herhangi bir eksik gözlem bulunmadığı
görülmektedir. Bu durum, tahsilat kayıtlarının yapısal olarak
tam ve tutarlı olduğunu göstermektedir.

Buna karşılık, tahsilat türlerine ait sütunlarda yüksek sayıda
eksik değer bulunduğu dikkat çekmektedir. **Nakit Tahsilat**
sütununda 636.470, **Mahsuben Tahsilat** sütununda 629.451 ve
**Kredi Kartı Tahsilatı** sütununda 636.993 eksik değer yer
almaktadır. **Banka Tahsilatı** sütununda ise yalnızca 8.060
eksik değer bulunmaktadır.

Bu yapı, her bir tahsilat kaydında yalnızca kullanılan ödeme
kanalına ait tutarın doldurulduğunu, diğer ödeme kanallarının
ise doğal olarak boş bırakıldığını göstermektedir. Dolayısıyla
bu eksik değerler veri kalitesine ilişkin bir problem değil,
tahsilat verisinin çoklu ödeme yöntemi yapısından kaynaklanan
beklenen bir durumdur.


In [11]:
df_tahsilat_1.head()

,Mali yıl/dönem,İl,İlçe,Söz.hsp.(bağımsız),Hesap Sınıfı,Tahakkuk Tutar,Son Ödeme Tarihinden Önceki Tahsilat,Son Ödeme Tarihindeki Tahsilat,Son Ödeme (1),Son Ödeme (2),...,Son Ödeme (5),Son Ödeme (6-10),Son Ödeme (10-20),Son Ödeme (20-30),Son Ödeme (30-60),Son Ödeme (60-90),Son Ödeme (90-120),Son Ödeme (120-150),Son Ödeme (150-180),Son Ödeme (180+)
0,OCK 2023,AMASYA,GÖYNÜCEK,9374624783,Mesken,5.03,0.03,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
1,OCK 2023,AMASYA,GÖYNÜCEK,236184905,Mesken,26.46,0.06,26.40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OCK 2023,AMASYA,GÖYNÜCEK,9657731015,Mesken,121.53,NaN,NaN,NaN,121.53,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,OCK 2023,AMASYA,GÖYNÜCEK,9554442880,Mesken,117.49,NaN,117.49,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,OCK 2023,AMASYA,GÖYNÜCEK,6031642522,Mesken,170.30,170.30,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
df_tahsilat_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 917632 entries, 0 to 917631
Data columns (total 22 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   Mali yıl/dönem                        917632 non-null  object 
 1   İl                                    917632 non-null  object 
 2   İlçe                                  917632 non-null  object 
 3   Söz.hsp.(bağımsız)                    917632 non-null  int64  
 4   Hesap Sınıfı                          917632 non-null  object 
 5   Tahakkuk Tutar                        917632 non-null  float64
 6   Son Ödeme Tarihinden Önceki Tahsilat  623908 non-null  float64
 7   Son Ödeme Tarihindeki Tahsilat        328193 non-null  float64
 8   Son Ödeme (1)                         20902 non-null   float64
 9   Son Ödeme (2)                         21664 non-null   float64
 10  Son Ödeme (3)                         18893 non-null   float64
 11  

In [104]:
df_tahsilat_1.value_counts("Mali yıl/dönem")

Mali yıl/dönem
TEM 2024    35717
AGU 2024    35239
EYL 2024    35034
EKM 2024    33955
EYL 2023    33588
AGU 2023    33511
TEM 2023    33468
HAZ 2024    33267
ARL 2023    33121
KSM 2024    32714
EKM 2023    32510
MAY 2025    32494
MAY 2023    32408
HAZ 2023    31863
NİS 2025    31699
NİS 2024    31625
MAY 2024    31606
KSM 2023    30762
ARL 2024    30717
OCK 2025    30385
MAR 2025    29863
ŞBT 2025    29522
NİS 2023    29371
MAR 2024    29326
MAR 2023    29178
ŞBT 2024    28945
OCK 2023    28874
OCK 2024    28784
ŞBT 2023    28086
Name: count, dtype: int64

In [159]:
# Kısaltılmış ay isimlerinin standart sayısal formatlara dönüştürülmesi için hazırlanan eşleştirme sözlüğü
ay_map = {
    "OCK": "01",
    "ŞBT": "02",
    "MAR": "03",
    "NİS": "04",
    "MAY": "05",
    "HAZ": "06",
    "TEM": "07",
    "AGU": "08",
    "EYL": "09",
    "EKM": "10",
    "KSM": "11",
    "ARL": "12"
}

In [160]:
# Metinsel mali dönem verilerinin standart tarih formatına dönüştürülmesi ve datetime tipine çevrimi
df_tahsilat_1["Mali yıl/dönem"] = pd.to_datetime(
    df_tahsilat_1["Mali yıl/dönem"]
      .str.strip()
      .str.split()
      .apply(lambda x: f"{x[1]}-{ay_map[x[0]]}-01")
)

**Not:** Tahsilat_1 verisinde yer alan **Mali yıl/dönem** değişkeni,
metinsel formatta ve ay–yıl bilgisini birlikte içerecek şekilde
tutulmaktadır. Bu yapı, zaman bazlı analizlerde ve sıralama
işlemlerinde doğrudan kullanıma uygun değildir.

Bu nedenle, ilgili değişken öncelikle metinsel olarak
temizlenmiş, ay ve yıl bilgileri ayrıştırılmış ve ay isimleri
önceden tanımlanan `ay_map` sözlüğü kullanılarak sayısal ay
değerlerine dönüştürülmüştür. Ardından, her dönem için ayın
ilk günü referans alınarak standart bir tarih formatı
(`YYYY-MM-01`) oluşturulmuş ve değişken `datetime` veri tipine
çevrilmiştir.

Bu dönüşüm sayesinde, tahsilat verileri zaman ekseninde
sıralanabilir hale gelmiş; aylık, dönemsel ve trend bazlı
analizlerin tutarlı ve hatasız biçimde yapılması sağlanmıştır.


In [121]:
# Hesap sınıflarına göre tahsilat_1 verilerinin dağılımı ve sayımı
df_tahsilat_1.value_counts(["Hesap Sınıfı"])

Hesap Sınıfı                            
Mesken                                      797842
Ticari Faaliyet - Yazıhane                   70001
Tarımsal Faaliyetler (Şahıs)                  9636
İbadethane Isıtma/Soğutma/Lojman              7637
Şantiye ve Geçici Aboneler                    6751
İbadethane Aydınlatma                         5322
Bina Ort Kul (Asn,Hidr,Kapıcı Dai vb.)        4167
1 SAYILI CETVELDE YER ALAN KAMU İDARESİ       3833
Şehit Aileleri ve Gaziler                     2912
Köy İçme Suyu Temini ve Dağıtımı Tesisi       1322
Belediye                                      1321
Büyükbaş-Küçükbaş Hayvancılık                 1121
Resmi Daire                                    965
Tarımsal Faaliyetler (Kooperatif)              929
Süt Toplama Merkezi                            608
İktisadi İşletmeleri Hariç Dernek ve Vkf       386
Resmi SAĞLIK KURULUŞLARI,RESMİ SPOR TES.       354
Cemevleri                                      308
Resmi Daire Lojman                       

In [161]:
# Belirlenen sütunların bellek verimliliği için kategori tipine dönüştürülmesi
Cat_sutunlar_1 = [
    "İl",
    "İlçe",
    "Hesap Sınıfı",
]

df_tahsilat_1[Cat_sutunlar_1] = df_tahsilat_1[Cat_sutunlar_1].apply(lambda x: x.astype("category"))

**Not:** *tahsilat_1* tablosundaki **İl**, **İlçe** ve **Hesap Sınıfı** kolonları
kategorik (categorical) veri tipine dönüştürülmüştür.

In [162]:
# Sözleşme hesap numaralarının metinsel veri tipine dönüştürülmesi
df_tahsilat_1["Söz.hsp.(bağımsız)"] = df_tahsilat_1["Söz.hsp.(bağımsız)"].astype(str)

**Not:** *tahsilat_1* tablosundaki **Söz.hsp.(bağımsız)** kolonu,
kimlik niteliği taşıması nedeniyle **string (object)** veri tipine dönüştürülmüştür.

In [140]:
df_tahsilat_1.describe().round(2).T

,count,mean,min,25%,50%,75%,max,std
Mali yıl/dönem,917632,2024-03-05 16:32:06.733156864,2023-01-01 00:00:00,2023-08-01 00:00:00,2024-03-01 00:00:00,2024-10-01 00:00:00,2025-05-01 00:00:00,NaN
Tahakkuk Tutar,917632.0,508.58,-12793.28,110.15,202.07,321.57,1173258.44,5052.67
Son Ödeme Tarihinden Önceki Tahsilat,623908.0,206.31,-12793.28,0.32,44.22,210.22,799298.89,2855.3
Son Ödeme Tarihindeki Tahsilat,328193.0,547.26,0.0,116.6,210.99,334.27,429056.57,3898.72
Son Ödeme (1),20902.0,643.06,-70.0,125.61,222.0,349.98,393238.0,5454.37
Son Ödeme (2),21664.0,553.71,-296.93,112.92,211.0,333.0,319120.0,5123.76
Son Ödeme (3),18893.0,414.44,-120.6,119.0,203.0,320.0,152377.0,2650.06
Son Ödeme (4),16995.0,536.36,0.0,116.9,208.07,329.86,560239.0,6242.19
Son Ödeme (5),7323.0,516.08,-752.0,115.0,218.54,344.98,158004.45,3451.49
Son Ödeme (6-10),45708.0,558.64,-100.66,121.9,213.59,330.0,550420.87,6666.26


Tahsilat_1 verisindeki mali yıl/dönem değişkeni incelendiğinde,
kayıtların **1 Ocak 2023 ile 1 Mayıs 2025 tarihleri arasını**
kapsadığı görülmektedir. 

Tahakkuk tutarları toplam 917.632 kayıt üzerinden incelenmiş olup,
ortalama tahakkuk tutarı 508,58 TL’dir. Medyan değerin 202,07 TL
olması, az sayıda çok yüksek tutarlı tahakkuk kaydının ortalamayı
yukarı çektiğini göstermektedir. Maksimum tahakkuk tutarının
1.173.258 TL’ye ulaşması ve standart sapmanın 5.052 TL olması,
tahakkuk tutarlarının dağılımının oldukça geniş olduğunu ortaya
koymaktadır.

Son ödeme tarihinden **önce yapılan tahsilatlarda 623.908 işlem**
bulunmakta olup, ortalama tahsilat tutarı 206,31 TL’dir. Medyan
değerin 44,22 TL seviyesinde olması, erken ödeme yapan
abonelerin çoğunlukla düşük tutarlı ödemeler gerçekleştirdiğini
göstermektedir.

**Son ödeme tarihinde yapılan tahsilatlar 328.193 işlem** ile
önemli bir paya sahiptir. Bu grupta ortalama tahsilat tutarı
547,26 TL, medyan tutar ise 210,99 TL’dir. Maksimum değerin
429.056 TL’ye ulaşması, son ödeme tarihinde yüksek tutarlı
ödemelerin yoğunlaştığını göstermektedir.

**Not:** Bazı kayıtların parçalı şekilde tahsis edildiği kayıtların işlem toplamından anlaşılmıştır.


In [175]:
df_tahsilat_1.isna().sum()

Mali yıl/dönem                               0
İl                                           0
İlçe                                         0
Söz.hsp.(bağımsız)                           0
Hesap Sınıfı                                 0
Tahakkuk Tutar                               0
Son Ödeme Tarihinden Önceki Tahsilat    293724
Son Ödeme Tarihindeki Tahsilat          589439
Son Ödeme (1)                           896730
Son Ödeme (2)                           895968
Son Ödeme (3)                           898739
Son Ödeme (4)                           900637
Son Ödeme (5)                           910309
Son Ödeme (6-10)                        871924
Son Ödeme (10-20)                       869351
Son Ödeme (20-30)                       888627
Son Ödeme (30-60)                       894602
Son Ödeme (60-90)                       910448
Son Ödeme (90-120)                      913812
Son Ödeme (120-150)                     915330
Son Ödeme (150-180)                     916081
Son Ödeme (18

Tahsilat_1 tablosundaki eksik değerler incelendiğinde,
kimlik ve temel tahakkuk bilgilerini içeren sütunlarda
(**Mali yıl/dönem**, **İl**, **İlçe**, **Söz.hsp.(bağımsız)**,
**Hesap Sınıfı** ve **Tahakkuk Tutar**) herhangi bir eksik
gözlem bulunmadığı görülmektedir. Bu durum, tahakkuk
kayıtlarının yapısal olarak tam ve analiz için uygun
olduğunu göstermektedir.

Buna karşılık, son ödeme tarihine göre ayrıştırılmış
tahsilat sütunlarında yüksek sayıda eksik değer yer
almaktadır. Özellikle **Son Ödeme (1)** ve sonraki
gecikme dilimlerinde eksik değer sayılarının belirgin
şekilde artması, tahsilatların büyük bir bölümünün
son ödeme tarihinden önce veya son ödeme tarihinde
gerçekleştiğine işaret etmektedir.

Örneğin, **Son Ödeme Tarihinden Önceki Tahsilat**
sütununda 293.724, **Son Ödeme Tarihindeki Tahsilat**
sütununda ise 589.439 eksik değer bulunmaktadır.
Buna karşın, **Son Ödeme (1) Tarihli Tahsilat** ve üzeri gecikmeli ödeme
dilimlerinde eksik değer sayılarının 890 binin üzerine
çıkması, bu gecikme aralıklarında gerçekleşen
tahsilat sayısının oldukça sınırlı olduğunu
göstermektedir.

Bu yapı, tahsilat_1 tablosunda yer alan eksik
değerlerin veri kalitesine ilişkin bir problem
olmadığını; tahsilatların zamanlama bazlı olarak
sınıflandırılmasından kaynaklanan, beklenen ve
anlamlı bir durum olduğunu göstermektedir.


In [175]:
# Toplam kayıt sayısı
toplam_kayit = df_tahsilat_1.shape[0]

# Zamanında veya erken ödeme:
# Son ödeme tarihinden önce veya son ödeme tarihinde tahsilat yapılan kayıtlar
zamaninda_veya_erken = df_tahsilat_1[
    (df_tahsilat_1["Son Ödeme Tarihinden Önceki Tahsilat"].notna()) |
    (df_tahsilat_1["Son Ödeme Tarihindeki Tahsilat"].notna())
]

zamaninda_veya_erken_sayi = zamaninda_veya_erken.shape[0]
zamaninda_veya_erken_oran = zamaninda_veya_erken_sayi / toplam_kayit * 100

print(
    f"Zamanında veya erken ödeme oranı: %{round(zamaninda_veya_erken_oran)} "
)

Zamanında veya erken ödeme oranı: %86 


In [176]:
# Geç ödeme:
# Son ödeme tarihinden sonraki herhangi bir gecikme diliminde tahsilat yapılan kayıtlar
gec_odeme_sutunlari = [
    "Son Ödeme (1)", "Son Ödeme (2)", "Son Ödeme (3)", "Son Ödeme (4)",
    "Son Ödeme (5)", "Son Ödeme (6-10)", "Son Ödeme (10-20)",
    "Son Ödeme (20-30)", "Son Ödeme (30-60)", "Son Ödeme (60-90)",
    "Son Ödeme (90-120)", "Son Ödeme (120-150)",
    "Son Ödeme (150-180)", "Son Ödeme (180+)"
]

gec_odeme = df_tahsilat_1[
    df_tahsilat_1[gec_odeme_sutunlari].notna().any(axis=1)
]

gec_odeme_sayi = gec_odeme.shape[0]
gec_odeme_oran = gec_odeme_sayi / toplam_kayit * 100

print(
    f"Geç ödeme oranı: %{round(gec_odeme_oran)} "
)

Geç ödeme oranı: %27 


Bu adımda, tahsilat kayıtları ödeme zamanlamasına göre
sınıflandırılmıştır. Son ödeme tarihinde veya öncesinde
tahsilat yapılan kayıtlar **zamanında/erken ödeme**,
son ödeme tarihinden sonra tahsilat yapılan kayıtlar ise
**geç ödeme** olarak tanımlanmış ve her iki grubun toplam
kayıtlar içindeki oranları hesaplanmıştır. Bazı kayıtlar hem zamanında hem geç ödeme **(kısmi ödemeler)** içerdiği orandalardan gözlemlenmiştir. 

In [74]:
df_hamamozu.head()

,il,ilce,sozlesme_hesap_no,mali_yil_donem,fatura_tarihi,kayit_tarihi,vade_tarihi,hesap_sinifi,Hesap Sınıfı,kwh
0,AMASYA,HAMAMÖZÜ,917576806,2023-01-01,2023-01-12,2023-03-06,2023-01-23,M001,Mesken,1.79
1,AMASYA,HAMAMÖZÜ,917576806,2023-01-01,2023-02-09,2023-05-11,2023-02-20,M001,Mesken,2.60
2,AMASYA,HAMAMÖZÜ,917576806,2023-02-01,2023-02-09,2023-05-11,2023-02-20,M001,Mesken,1.23
3,AMASYA,HAMAMÖZÜ,917576806,2023-02-01,2023-03-10,2023-05-11,2023-03-20,M001,Mesken,2.56
4,AMASYA,HAMAMÖZÜ,917576806,2023-03-01,2023-03-10,2023-05-11,2023-03-20,M001,Mesken,1.35


In [75]:
df_hamamozu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124818 entries, 0 to 124817
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   il                 124818 non-null  object 
 1   ilce               124818 non-null  object 
 2   sozlesme_hesap_no  124818 non-null  int64  
 3   mali_yil_donem     124818 non-null  object 
 4   fatura_tarihi      124818 non-null  object 
 5   kayit_tarihi       124818 non-null  object 
 6   vade_tarihi        124818 non-null  object 
 7   hesap_sinifi       124818 non-null  object 
 8   Hesap Sınıfı       124818 non-null  object 
 9   kwh                124818 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 9.5+ MB


In [141]:
# Hesap_sinifi'na göre Hamamozu verilerinin dağılımı ve sayımı
df_hamamozu.value_counts("hesap_sinifi")

hesap_sinifi
M001    110682
T001      7468
T019      1510
M002      1443
TA01       661
A001       528
T021       495
T002       402
M003       177
T013       177
3          149
4          147
T007       127
TA03       127
SE01       120
T009       113
TA04       107
T020        76
T010        63
S001        46
T008        46
T012        40
M004        33
M006        31
T006        28
TA02        14
T017         5
6            3
Name: count, dtype: int64

In [203]:
df_hamamozu["hesap_sinifi"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 124818 entries, 0 to 124817
Series name: hesap_sinifi
Non-Null Count   Dtype   
--------------   -----   
124818 non-null  category
dtypes: category(1)
memory usage: 123.3 KB


In [162]:
df_hamamozu.describe().T

,count,mean,min,25%,50%,75%,max,std
mali_yil_donem,124818,2024-03-29 20:51:13.402874368,2023-01-01 00:00:00,2023-08-01 00:00:00,2024-04-01 00:00:00,2024-12-01 00:00:00,2025-07-01 00:00:00,NaN
fatura_tarihi,124818,2024-05-16 08:20:16.901408512,2023-01-04 00:00:00,2023-09-12 00:00:00,2024-05-15 00:00:00,2025-01-13 00:00:00,2025-10-02 00:00:00,NaN
kayit_tarihi,124818,2024-07-17 02:09:53.568235264,2023-03-06 00:00:00,2023-11-28 00:00:00,2024-07-13 00:00:00,2025-03-13 00:00:00,2025-10-09 00:00:00,NaN
vade_tarihi,124818,2024-05-27 02:57:33.771091200,2023-01-06 00:00:00,2023-09-22 00:00:00,2024-05-27 00:00:00,2025-01-23 00:00:00,2025-10-09 00:00:00,NaN
kwh,124818.0,70.874619,-1242.99,15.49,40.56,70.43,25941.6,389.217875


Mali yıl/dönem değişkeni 1 Ocak 2023
ile 1 Temmuz 2025 tarihleri arasını kapsamaktadır.

Fatura tarihi incelendiğinde, işlemlerin 4 Ocak 2023 ile
2 Ekim 2025 tarihleri arasında gerçekleştiği görülmektedir.

Kayıt tarihi değişkeni, fatura tarihinden sonraki süreci
yansıtmakta olup, medyan kayıt tarihinin 13 Temmuz 2024 olması,
işlemlerin sistemsel olarak belirli bir gecikme ile kayda
alındığını göstermektedir.

**Hamamözü** için kWh tüketim değerleri **124.818 kayıt** üzerinden incelenmiş olup,
**ortalama tüketim 70,87 kWh**, medyan tüketim ise 40,56 kWh
seviyesindedir. Ortalama değerin medyana kıyasla yüksek olması,
tüketim dağılımının sağa çarpık olduğunu göstermektedir.
Minimum değerin -1.242,99 kWh, maksimum değerin ise 25.941,6 kWh
olması, veri setinde hem düzeltme/iptal kaynaklı negatif
değerlerin hem de çok yüksek tüketimli uç değerlerin bulunduğunu
ortaya koymaktadır.

Standart sapmanın 389,22 kWh gibi yüksek bir değerde olması,
tüketim miktarlarının oldukça heterojen olduğunu ve aboneler
arasında belirgin tüketim farkları bulunduğunu göstermektedir.


In [176]:
df_hamamozu.isna().sum()

il                   0
ilce                 0
sozlesme_hesap_no    0
mali_yil_donem       0
fatura_tarihi        0
kayit_tarihi         0
vade_tarihi          0
hesap_sinifi         0
Hesap Sınıfı         0
kwh                  0
dtype: int64

Eksik veri tespit edilmemiştir.

In [115]:
df_gumushacikoy.head()

,il,ilce,sozlesme_hesap_no,mali_yil_donem,fatura_tarihi,kayit_tarihi,vade_tarihi,hesap_sinifi,Hesap Sınıfı,kwh
0,AMASYA,GÜMÜŞHACIKÖY,7444449517,2023-01-01,2023-01-11,2023-03-06,2023-01-23,M001,Mesken,21.85
1,AMASYA,GÜMÜŞHACIKÖY,7444449517,2023-01-01,2023-02-10,2023-05-11,2023-02-20,M001,Mesken,44.50
2,AMASYA,GÜMÜŞHACIKÖY,7444449517,2023-02-01,2023-02-10,2023-05-11,2023-02-20,M001,Mesken,22.25
3,AMASYA,GÜMÜŞHACIKÖY,7444449517,2023-02-01,2023-03-10,2023-05-11,2023-03-20,M001,Mesken,45.71
4,AMASYA,GÜMÜŞHACIKÖY,7444449517,2023-03-01,2023-03-10,2023-05-11,2023-03-20,M001,Mesken,25.40


In [80]:
df_gumushacikoy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 765657 entries, 0 to 765656
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   il                 765657 non-null  object 
 1   ilce               765657 non-null  object 
 2   sozlesme_hesap_no  765657 non-null  int64  
 3   mali_yil_donem     765657 non-null  object 
 4   fatura_tarihi      765657 non-null  object 
 5   kayit_tarihi       765657 non-null  object 
 6   vade_tarihi        765657 non-null  object 
 7   hesap_sinifi       765657 non-null  object 
 8   Hesap Sınıfı       765657 non-null  object 
 9   kwh                765657 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 58.4+ MB


In [196]:
df_gumushacikoy.describe().T

,count,mean,min,25%,50%,75%,max,std
mali_yil_donem,765657,2024-03-30 10:03:51.584638720,2023-01-01 00:00:00,2023-08-01 00:00:00,2024-04-01 00:00:00,2024-12-01 00:00:00,2025-07-01 00:00:00,NaN
fatura_tarihi,765657,2024-05-19 07:31:50.382847744,2023-01-02 00:00:00,2023-09-19 00:00:00,2024-05-22 00:00:00,2025-01-13 00:00:00,2025-10-01 00:00:00,NaN
kayit_tarihi,765657,2024-07-15 19:18:33.871616,2023-03-06 00:00:00,2023-11-28 00:00:00,2024-07-13 00:00:00,2025-03-13 00:00:00,2025-10-09 00:00:00,NaN
vade_tarihi,765657,2024-05-29 22:02:55.444618496,2023-01-03 00:00:00,2023-09-29 00:00:00,2024-06-03 00:00:00,2025-01-23 00:00:00,2025-10-20 00:00:00,NaN
kwh,765657.0,97.336632,-25370.64,18.57,48.31,82.72,153575.73,1077.758336


Mali yıl/dönem değişkeni ** Ocak 2023 ile
Temmuz 2025** tarihleri arasını kapsamaktatır.

**Gümüşhacıköy** kWh tüketim değerleri **765.657 kayıt** üzerinden incelenmiş olup,
**ortalama tüketim 97,34 kWh**, medyan tüketim ise 48,31 kWh
seviyesindedir. 

Minimum tüketim değerinin -25.370,64 kWh, maksimum tüketim
değerinin ise 153.575,73 kWh olması, veri setinde düzeltme veya
iptal işlemlerinden kaynaklanan negatif değerlerin yanı sıra,
çok yüksek tüketimli uç kayıtların da bulunduğunu göstermektedir.
Standart sapmanın 1.077,76 kWh gibi oldukça yüksek bir seviyede
olması, aboneler arasında tüketim davranışlarının son derece
heterojen olduğunu ortaya koymaktadır.


In [197]:
df_gumushacikoy.isna().sum()

il                   0
ilce                 0
sozlesme_hesap_no    0
mali_yil_donem       0
fatura_tarihi        0
kayit_tarihi         0
vade_tarihi          0
hesap_sinifi         0
Hesap Sınıfı         0
kwh                  0
dtype: int64

Eksik veri tespit edilmemiştir.

In [116]:
df_goynucek.head()

,il,ilce,sozlesme_hesap_no,mali_yil_donem,fatura_tarihi,kayit_tarihi,vade_tarihi,hesap_sinifi,Hesap Sınıfı,kwh
0,AMASYA,GÖYNÜCEK,9374624783,2023-01-01,2023-01-14,2023-03-06,2023-01-24,M001,Mesken,0.10
1,AMASYA,GÖYNÜCEK,9374624783,2023-01-01,2025-03-12,2025-05-09,2025-03-24,M001,Mesken,0.12
2,AMASYA,GÖYNÜCEK,9374624783,2023-02-01,2025-03-12,2025-05-09,2025-03-24,M001,Mesken,0.20
3,AMASYA,GÖYNÜCEK,9374624783,2023-03-01,2025-03-12,2025-05-09,2025-03-24,M001,Mesken,0.00
4,AMASYA,GÖYNÜCEK,9374624783,2023-04-01,2025-03-12,2025-05-09,2025-03-24,M001,Mesken,0.08


In [78]:
df_goynucek.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295223 entries, 0 to 295222
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   il                 295223 non-null  object 
 1   ilce               295223 non-null  object 
 2   sozlesme_hesap_no  295223 non-null  int64  
 3   mali_yil_donem     295223 non-null  object 
 4   fatura_tarihi      295223 non-null  object 
 5   kayit_tarihi       295223 non-null  object 
 6   vade_tarihi        295223 non-null  object 
 7   hesap_sinifi       295223 non-null  object 
 8   Hesap Sınıfı       295223 non-null  object 
 9   kwh                295223 non-null  float64
dtypes: float64(1), int64(1), object(8)
memory usage: 22.5+ MB


In [163]:
df_goynucek.describe().T

,count,mean,min,25%,50%,75%,max,std
mali_yil_donem,295223,2024-03-31 07:10:50.376156416,2023-01-01 00:00:00,2023-08-01 00:00:00,2024-04-01 00:00:00,2024-12-01 00:00:00,2025-07-01 00:00:00,NaN
fatura_tarihi,295223,2024-05-16 11:46:37.086947840,2023-01-02 00:00:00,2023-09-14 00:00:00,2024-05-17 00:00:00,2025-01-14 00:00:00,2025-10-03 00:00:00,NaN
kayit_tarihi,295223,2024-07-15 16:29:42.495266560,2023-03-06 00:00:00,2023-11-28 00:00:00,2024-07-13 00:00:00,2025-03-13 00:00:00,2025-10-09 00:00:00,NaN
vade_tarihi,295223,2024-05-27 05:13:04.682765312,2023-01-03 00:00:00,2023-09-25 00:00:00,2024-05-27 00:00:00,2025-01-24 00:00:00,2025-10-13 00:00:00,NaN
kwh,295223.0,89.669891,-4208.64,17.86,45.09,77.14,105687.69,742.276369


Mali yıl/dönem değişkeni **Ocak 2023 ile
Temmuz 2025** tarihleri arasını kapsamaktatır.

**Göynücek** kWh tüketim değerleri **295.223 kayıt** üzerinden incelenmiş olup,
**ortalama tüketim 89,67 kWh**, medyan tüketim ise 45,09 kWh
seviyesindedir. Ortalama değerin medyana kıyasla yüksek olması,
tüketim dağılımının belirgin şekilde sağa çarpık olduğunu
göstermektedir.

In [180]:
df_goynucek.isna().sum()

il                   0
ilce                 0
sozlesme_hesap_no    0
mali_yil_donem       0
fatura_tarihi        0
kayit_tarihi         0
vade_tarihi          0
hesap_sinifi         0
Hesap Sınıfı         0
kwh                  0
dtype: int64

Eksik veri tespit edilmemiştir.

In [31]:
# İlçe bazlı veri setlerindeki tekil (benzersiz) müşteri sayılarının tespiti
print(
    f"Hamamözü benzersiz müşteri sayısı: {df_hamamozu['sozlesme_hesap_no'].nunique()}\n"
    f"Gümüşhacıköy benzersiz müşteri sayısı: {df_gumushacikoy['sozlesme_hesap_no'].nunique()}\n"
    f"Göynücek benzersiz müşteri sayısı: {df_goynucek['sozlesme_hesap_no'].nunique()}"
)

Hamamözü benzersiz müşteri sayısı: 2981
Gümüşhacıköy benzersiz müşteri sayısı: 18190
Göynücek benzersiz müşteri sayısı: 7128


### Veri Setlerinin Birleştirilmesi

Bu aşamada aynı yapıya sahip, farklı dönemlere ait tablolar
tek bir veri seti altında birleştirilmiştir.

Birleştirme işlemi sırasında:
- Sütun isimlerinin tutarlılığı korunmuş,
- Veri tipleri uyumlu hâle getirilmiş,
- Kayıt bütünlüğünü bozacak herhangi bir filtreleme yapılmamıştır.

Amaç, analizde kullanılacak tek ve tutarlı bir ana tablo oluşturmaktır.

In [163]:
# İlçe bazlı ayrıştırılmış veri setlerinin tek bir ana tabloda birleştirilmesi
df_tahakkuk_b = pd.concat([df_hamamozu, df_gumushacikoy, df_goynucek], ignore_index=True)

In [164]:
# Birleştirilen ana tahakkuk veri setinin toplam satır ve sütun sayısının kontrolü
df_tahakkuk_b.shape

(1185698, 10)

In [211]:
df_tahakkuk_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1185698 entries, 0 to 1185697
Data columns (total 10 columns):
 #   Column             Non-Null Count    Dtype         
---  ------             --------------    -----         
 0   il                 1185698 non-null  category      
 1   ilce               1185698 non-null  object        
 2   sozlesme_hesap_no  1185698 non-null  object        
 3   mali_yil_donem     1185698 non-null  datetime64[ns]
 4   fatura_tarihi      1185698 non-null  datetime64[ns]
 5   kayit_tarihi       1185698 non-null  datetime64[ns]
 6   vade_tarihi        1185698 non-null  datetime64[ns]
 7   hesap_sinifi       1185698 non-null  object        
 8   Hesap Sınıfı       1185698 non-null  object        
 9   kwh                1185698 non-null  float64       
dtypes: category(1), datetime64[ns](4), float64(1), object(4)
memory usage: 82.5+ MB


In [165]:
# Belirlenen sütunların bellek verimliliği için kategori tipine dönüştürülmesi
Cat_sutunlar_2 = [
    "il",
    "ilce",
    "Hesap Sınıfı",
    "hesap_sinifi"
]

df_tahakkuk_b[Cat_sutunlar_2] = df_tahakkuk_b[Cat_sutunlar_2].apply(lambda x: x.astype("category"))

**Not:** *tahakkuk_b* tablosundaki **İl**, **İlçe**, **Hesap Sınıfı** ve **hesap_sinifi** kolonları
kategorik (categorical) veri tipine dönüştürülmüştür.

In [166]:
# Sözleşme hesap numaralarının metinsel veri tipine dönüştürülmesi
df_tahakkuk_b["sozlesme_hesap_no"] = df_tahakkuk_b["sozlesme_hesap_no"].astype(str)

**Not:** *Tahakkuk_b* tablosundaki **sozlesme_hesap_no** kolonu,
kimlik niteliği taşıması nedeniyle **int** veri tipinden **object** veri tipine dönüştürülmüştür.

In [167]:
# Belirlenen tarih sütunlarının standart datetime formatına toplu dönüşümü
tarih_sutunlari = [
    "mali_yil_donem",
    "fatura_tarihi",
    "kayit_tarihi",
    "vade_tarihi"
]

df_tahakkuk_b[tarih_sutunlari] = df_tahakkuk_b[tarih_sutunlari].apply(pd.to_datetime)

**Not:** *Tahakkuk_b* tablosunda yer alan tarih sütunları,
zaman bazlı analizlerin sağlıklı bir şekilde gerçekleştirilebilmesi
amacıyla **object** veri tipinden **datetime** veri tipine
dönüştürülmüştür.


In [35]:
df_tahakkuk_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124818 entries, 0 to 124817
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   il                 124818 non-null  category      
 1   ilce               124818 non-null  category      
 2   sozlesme_hesap_no  124818 non-null  object        
 3   mali_yil_donem     124818 non-null  datetime64[ns]
 4   fatura_tarihi      124818 non-null  datetime64[ns]
 5   kayit_tarihi       124818 non-null  datetime64[ns]
 6   vade_tarihi        124818 non-null  datetime64[ns]
 7   hesap_sinifi       124818 non-null  category      
 8   Hesap Sınıfı       124818 non-null  category      
 9   kwh                124818 non-null  float64       
dtypes: category(4), datetime64[ns](4), float64(1), object(1)
memory usage: 6.2+ MB


In [217]:
df_tahakkuk_b.describe().T.round(2)

,count,mean,min,25%,50%,75%,max,std
mali_yil_donem,1185698,2024-03-30 13:55:52.818002944,2023-01-01 00:00:00,2023-08-01 00:00:00,2024-04-01 00:00:00,2024-12-01 00:00:00,2025-07-01 00:00:00,NaN
fatura_tarihi,1185698,2024-05-18 07:09:59.300666368,2023-01-02 00:00:00,2023-09-19 00:00:00,2024-05-22 00:00:00,2025-01-13 00:00:00,2025-10-03 00:00:00,NaN
kayit_tarihi,1185698,2024-07-15 21:51:24.613449728,2023-03-06 00:00:00,2023-11-28 00:00:00,2024-07-13 00:00:00,2025-03-13 00:00:00,2025-10-09 00:00:00,NaN
vade_tarihi,1185698,2024-05-28 22:50:39.356058880,2023-01-03 00:00:00,2023-09-29 00:00:00,2024-06-03 00:00:00,2025-01-23 00:00:00,2025-10-20 00:00:00,NaN
kwh,1185698.0,92.642069,-25370.64,18.01,46.62,80.0,153575.73,950.405222


Üç ilçeyenin **birleştirilmiş tahakkük veri setinde**Mali yıl/dönem değişkeni **Ocak 2023 ile
Temmuz 2025** tarihleri arasını kapsamaktadır.

**birleştirilmiş tahakkük veri setinde** kWh tüketim değerleri **1.185.698 kayıt** üzerinden incelenmiş olup,
**ortalama tüketim 92,64 kWh**, medyan tüketim ise 46,62 kWh
seviyesindedir. Ortalama değerin medyana kıyasla yüksek olması,
tüketim dağılımının belirgin şekilde sağa çarpık olduğunu
göstermektedir. Birinci ve üçüncü çeyrek değerlerinin
18,01 kWh ve 80,00 kWh olması, kayıtların büyük bir bölümünün
düşük ve orta tüketim aralıklarında yoğunlaştığını ortaya
koymaktadır.

Minimum tüketim değerinin -25.370,64 kWh, maksimum tüketim
değerinin ise 153.575,73 kWh olması, veri setinde düzeltme veya
iptal işlemlerinden kaynaklanan negatif değerlerin yanı sıra,
çok yüksek tüketimli uç kayıtların da bulunduğunu göstermektedir.
Standart sapmanın 950,41 kWh gibi oldukça yüksek bir seviyede
olması, aboneler arasında tüketim davranışlarının son derece
heterojen olduğunu ortaya koymaktadır.


In [220]:
df_tahakkuk_b.isna().sum()

il                   0
ilce                 0
sozlesme_hesap_no    0
mali_yil_donem       0
fatura_tarihi        0
kayit_tarihi         0
vade_tarihi          0
hesap_sinifi         0
Hesap Sınıfı         0
kwh                  0
dtype: int64

Eksik veri tespit edilmemiştir.

### Negatif Tüketim Değerlerinin Değerlendirilmesi

Veri setinde negatif kWh değerleri tespit edilmiştir.
Bu değerler, iptal veya düzeltme faturalarından kaynaklanmakta olup
veri hatası olarak değerlendirilmemiştir.

Söz konusu kayıtlar analizden çıkarılmamış,
ancak yorumlama aşamasında iş kuralı çerçevesinde
dikkate alınmıştır.

In [170]:
# Tahakkuk verilerindeki kWh değerlerinin negatif ve sıfır olma durumlarına göre anomalilerin tespiti
print(
    f"Negatif kWh Sayısı : {(df_tahakkuk_b['kwh'] < 0).sum()} \n"
    f"Sıfır kWh Sayısı: {(df_tahakkuk_b['kwh'] == 0).sum()} "
)
print("-"*50)
print(
    f"Negatif kWh Oranı  : %{round(((df_tahakkuk_b['kwh'] < 0).sum())/len(df_tahakkuk_b) * 100, 2)} \n"
    f"Sıfır kWh Oranı: %{round(((df_tahakkuk_b['kwh'] == 0).sum())/len(df_tahakkuk_b) * 100, 2)} "
)

Negatif kWh Sayısı : 151 
Sıfır kWh Sayısı: 55377 
--------------------------------------------------
Negatif kWh Oranı  : %0.01 
Sıfır kWh Oranı: %4.67 


In [229]:
# Tüketim değeri (kWh) negatif olan kayıtların incelenmesi
df_tahakkuk_b[df_tahakkuk_b["kwh"] < 0 ].head()

,il,ilce,sozlesme_hesap_no,mali_yil_donem,fatura_tarihi,kayit_tarihi,vade_tarihi,hesap_sinifi,Hesap Sınıfı,kwh
23573,AMASYA,HAMAMÖZÜ,7904741733,2025-03-01,2025-04-14,2025-06-10,2025-04-22,M001,Mesken,-1242.99
23576,AMASYA,HAMAMÖZÜ,7904741733,2025-04-01,2025-04-14,2025-06-10,2025-04-22,M001,Mesken,-733.04
28413,AMASYA,HAMAMÖZÜ,469034964,2023-10-01,2023-10-11,2023-11-28,2023-10-23,M001,Mesken,-1.04
70626,AMASYA,HAMAMÖZÜ,7722066065,2023-10-01,2023-10-23,2023-11-28,2023-11-02,M001,Mesken,-1.34
180501,AMASYA,GÜMÜŞHACIKÖY,2945215788,2024-02-01,2024-04-15,2024-06-15,2024-03-25,M001,Mesken,-374.09


In [68]:
# İlçe bazında ortalama kWh tüketim değerlerinin karşılaştırılması
df_tahakkuk_b.groupby("ilce", observed = False)["kwh"].agg( ortalama_kwh= "mean")

,ortalama_kwh
ilce,
GÖYNÜCEK,89.669891
GÜMÜŞHACIKÖY,97.336632
HAMAMÖZÜ,70.874619


Üç ilçenin karşılaştırılması gerçekleştirilmiştir.

### IQR Yöntemi
kWh değişkeni için IQR yöntemi kullanılarak uç değer analizi yapılmıştır.

In [172]:
# IQR yöntemi kullanılarak kWh tüketim verilerindeki aykırı değerlerin (outliers) tespiti ve sayılması
Q1 = df_tahakkuk_b["kwh"].quantile(0.25)
Q3 = df_tahakkuk_b["kwh"].quantile(0.75)
IQR = Q3 - Q1

alt_sinir = Q1 - 1.5 * IQR
ust_sinir = Q3 + 1.5 * IQR

outliers = df_tahakkuk_b[
    (df_tahakkuk_b["kwh"] < alt_sinir) | 
    (df_tahakkuk_b["kwh"] > ust_sinir)
]["kwh"]

print(
    f" Tahakkuk_b veri setindeki outlier sayısı: {outliers.shape[0] } "
)


 Tahakkuk_b veri setindeki outlier sayısı: 48554   


In [241]:
# Tespit edilen aykırı değerlerin en düşük seviyeden başlayarak sıralanması ve ilk örneklerin incelenmesi
outliers.sort_values().head(10)

642557    -25370.64
614593    -12574.78
408766     -9069.13
398719     -6851.44
1113942    -4208.64
866036     -3861.20
1151085    -2836.64
866037     -2332.63
866030     -2315.57
719646     -1981.66
Name: kwh, dtype: float64

In [242]:
# Tespit edilen aykırı değerlerin en yüksek seviyeden başlayarak sıralanması ve ilk örneklerin incelenmesi
outliers.sort_values().tail(10)

721103    109344.06
721105    109546.29
808468    111801.06
808472    116408.88
808458    119221.20
808448    124008.57
808464    134681.40
808470    136527.93
808466    150757.74
808462    153575.73
Name: kwh, dtype: float64

### Hesap Sınıfına Göre Toplulaştırma Analizi

Bu bölümde elektrik tüketimi değerleri,
hesap sınıfları bazında gruplanarak
ortalama, medyan ve standart sapma tüketim metrikleri hesaplanmıştır.

Bu analiz, farklı müşteri segmentlerinin
tüketim davranışlarını karşılaştırmalı olarak
değerlendirebilmek amacıyla gerçekleştirilmiştir.

In [50]:
# Tahakkuk tablosundan hesap sınıfına göre istatiksel değerlere ulaşma
hesap_sinifi_tablo = (  
    df_tahakkuk_b
    .groupby("Hesap Sınıfı", observed = False)["kwh"]
    .agg(
        ortalama_kwh="mean",
        medyan_kwh="median",
        std_kwh="std"
    )
    .reset_index()
)

hesap_sinifi_tablo.head(6)

,Hesap Sınıfı,ortalama_kwh,medyan_kwh,std_kwh
0,1 SAYILI CETVELDE YER ALAN KAMU İDARESİ,688.441598,23.860,3911.912974
1,Aritma Tesisleri,16594.174857,16186.910,11656.705924
2,Balıkçılık ve Su Ürünleri Yetiştiriciliğ,85.358306,79.375,42.120066
3,Belediye,600.581235,78.270,3350.978976
4,Belediye Park Bahçe Aydınlatma,31.951003,18.390,38.472449
5,"Bina Ort Kul (Asn,Hidr,Kapıcı Dai vb.)",29.965663,15.760,41.331395


In [47]:
# Tahakkuk tablosundan hesap sınıfına göre istatiksel değerlere ulaşma ( Sıralı - çoktan aza )
hesap_sinifi_tablo.sort_values( 
    by="ortalama_kwh",
    ascending=False
).round(1).head(6)

,Hesap Sınıfı,ortalama_kwh,medyan_kwh,std_kwh
11,Karayolları Genel Müdürlüğü Aydınlatma,30203.4,36810.9,16963.6
1,Aritma Tesisleri,16594.2,16186.9,11656.7
13,Lisansız Üreticiler,16155.3,322.3,35926.4
23,Sanayi,7293.8,1531.7,13716.3
33,İçme-Kullanma Suyu (Belediye),5213.5,400.0,8948.1
25,Tarımsal Faaliyetler (Kooperatif),3779.3,282.7,9426.9


Hesap sınıfı bazında kWh tüketim istatistikleri incelendiğinde,
yüksek ortalama tüketime sahip sınıfların aynı zamanda
yüksek değişkenlik sergilediği görülmektedir.

**Karayolları Genel Müdürlüğü Aydınlatma** sınıfında ortalama tüketim
30.203 kWh, medyan tüketim ise 36.811 kWh seviyesindedir.
Medyan değerin ortalamadan yüksek olması, bu sınıfta tüketimin
genel olarak yüksek seviyelerde yoğunlaştığını göstermektedir.
Standart sapmanın 16.964 kWh olması, tüketim değerlerinin
geniş bir aralığa yayıldığını ortaya koymaktadır.

**Arıtma Tesisleri** sınıfında ortalama tüketim 16.594 kWh,
medyan tüketim ise 16.187 kWh olarak hesaplanmıştır.
Ortalama ve medyan değerlerin birbirine yakın olması,
bu hesap sınıfında tüketim davranışının görece homojen
olduğunu göstermektedir. Standart sapma 11.657 kWh’tir.

**Lisansız Üreticiler** sınıfında ortalama tüketim 16.155 kWh
olmasına rağmen, medyan tüketimin yalnızca 322 kWh olması,
bu sınıfta çok sayıda düşük tüketimli kayıt ile az sayıda
çok yüksek tüketimli kaydın bir arada bulunduğunu göstermektedir.
Standart sapmanın 35.926 kWh gibi oldukça yüksek bir seviyede
olması, bu sınıfın dağılımının son derece çarpık olduğunu
ve güçlü outlier etkisi barındırdığını ortaya koymaktadır.

**Sanayi** abonelerinde ortalama tüketim 7.294 kWh, medyan tüketim
1.532 kWh seviyesindedir. Ortalama ve medyan arasındaki fark,
sanayi sınıfında yüksek tüketimli bazı abonelerin ortalamayı
yukarı çektiğini göstermektedir. Standart sapmanın 13.716 kWh
olması, tüketim davranışlarının oldukça heterojen olduğunu
ortaya koymaktadır.

**İçme-Kullanma Suyu (Belediye)** hesap sınıfında ortalama tüketim
5.214 kWh, medyan tüketim ise 400 kWh olarak hesaplanmıştır.
Bu fark, az sayıda yüksek tüketimli tesisin genel dağılımı
önemli ölçüde etkilediğini göstermektedir.

**Tarımsal Faaliyetler (Kooperatif)** sınıfında ortalama tüketim
3.779 kWh, medyan tüketim 283 kWh seviyesindedir.
Standart sapmanın 9.427 kWh olması, özellikle mevsimsel
kullanıma bağlı olarak tüketim değerlerinin ciddi dalgalanma
gösterdiğine işaret etmektedir.


In [143]:
# Tahakkuk tablosundan hesap sınıfına göre kayıt adetlerine ulaşma ( sıralı çoktan aza )
df_tahakkuk_b.groupby("Hesap Sınıfı", observed = True).size().sort_values(ascending=False).head(6)

Hesap Sınıfı
Mesken                                     1026609
Ticari Faaliyet - Yazıhane                   91298
Tarımsal Faaliyetler (Şahıs)                 17266
İbadethane Isıtma/Soğutma/Lojman             10173
1 SAYILI CETVELDE YER ALAN KAMU İDARESİ       8500
Şantiye ve Geçici Aboneler                    6706
dtype: int64

In [125]:
# Tahakkuk tablosundan hesap sınıfların toplam kayıttaki yüzdelerine ulaşma ( belirli sınıflar )
Hesap_Sınıfı_ilk_3 = ["Mesken","Ticari Faaliyet - Yazıhane","Tarımsal Faaliyetler (Şahıs)"]

for i in Hesap_Sınıfı_ilk_3 :
 print(
    f"{i} oranı : {(df_tahakkuk_b[df_tahakkuk_b["Hesap Sınıfı"] == i ]["kwh"].count()/len(df_tahakkuk_b) * 100).round(1)}"
 )

Mesken oranı : 86.6
Ticari Faaliyet - Yazıhane oranı : 7.7
Tarımsal Faaliyetler (Şahıs) oranı : 1.5


Hesap sınıfına göre tüketim istatistikleri incelendiğinde,
mesken abonelerinin ortalama tüketiminin görece düşük olduğu,
buna karşın sanayi, tarımsal ve kamu abonelerinin ortalama ve
medyan tüketimlerinin oldukça yüksek olduğu görülmektedir.

Standart sapmanın yüksek olması, hesap sınıfları içerisinde
tüketim davranışlarının heterojen olduğunu göstermektedir.

### Notebook’lar Arası Veri Aktarımı

Bu notebook’ta gerçekleştirilen veri temizleme, dönüştürme ve birleştirme
işlemleri sonucunda elde edilen nihai veri seti,
sonraki analiz adımlarında kullanılmak üzere dosya olarak dışa aktarılmıştır.

Veri setinin diske kaydedilmesinin temel amacı,
farklı notebook’lar arasında çalışmayı mümkün kılmak ve
analiz sürecini modüler bir yapıya ayırmaktır.

In [253]:
df_tahsilat.to_parquet("df_tahsilat_clean.parquet", index=False)

In [252]:
df_tahsilat_1.to_parquet("df_tahsilat_1_clean.parquet", index=False)

In [ ]:
# İndirme sırasında yaşanan problemden kaynaklı hesap_sinifi object data dönüştürülmüştür.
df_tahakkuk_b["hesap_sinifi"] = df_tahakkuk["hesap_sinifi"].astype(str) 

In [249]:
df_tahakkuk_b.to_parquet("df_tahakkuk_clean.parquet", index=False)

### Ara Değerlendirme

Bu bölümde veri setinin yapısı, kalite durumu ve temel
istatistiksel özellikleri incelenmiştir.